# Cross Entropy Method (Numpy)

Can run only on CPU.
Faster when number of samples is small.

Import libraries.

In [ ]:
%matplotlib notebook

%load_ext line_profiler

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ERROR! Session/line number was not unique in database. History logging moved to new session 180


2022-03-14 16:28:59.057367: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Num GPUs Available:  1


2022-03-14 16:28:59.600632: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-14 16:28:59.600988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-14 16:28:59.644524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-14 16:28:59.644902: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 849.46GiB/s
2022-03-14 16:28:59.644914: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-14 16:28:59.645999: I tensorflow/stream_executor/platform

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 13 17:19:13 2021

@author: edgar
"""

import numpy as np
from matplotlib import pyplot as plt

import time
start_time = time.time()

In [4]:
# Define a starting state of the system
history = 50

x0 = np.array([-1.01, 0,0,0 ]) # (Brendan 1) should be 4D to include a history of control 
print(x0,'\n')
x0 = np.repeat(x0[np.newaxis],history,axis=0)   # (Brendan 2) initalise with a history (will need to be modified to use motion capture)
print(x0)

# Define a target
xt          = np.array([0, 0]) #soft pendulum

time_step   = 100
delta_time  = 1/20#0.25
trials      = 50
horizon_num = 15
sample_num  = 5000
elite_limit = 10
epsilon     = 0.1
alpha_mean  = 0.1
alpha_var = 0.1

[-1.01  0.    0.    0.  ] 

[[-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]
 [-1.01  0.    0.    0.  ]


Define functions to update cart position:

In [ ]:
import tensorflow as tf
from tensorflow import keras

model_type ='../../../models/keras_models/LSTM/V9_Combinations_2'
model = keras.models.load_model(model_type,compile=False)

In [ ]:
model.summary()

In [ ]:
# Freeze the graph to use it with tensorflow
# sources
# https://stackoverflow.com/questions/60974077/how-to-save-keras-model-as-frozen-graph
# https://stackoverflow.com/questions/68656751/how-to-speed-up-the-keras-model-predict
# https://leimao.github.io/blog/Save-Load-Inference-From-TF2-Frozen-Graph/

from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()


# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    print("-" * 50)
    print("Frozen model layers: ")
    layers = [op.name for op in import_graph.get_operations()]
    if print_graph == True:
        for layer in layers:
            print(layer)
    print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

# Load frozen graph using TensorFlow 1.x functions
with tf.io.gfile.GFile("./frozen_models/frozen_graph.pb", "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

# Wrap frozen graph to ConcreteFunctions
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["x:0"],
                                outputs=["Identity:0"],
                                print_graph=True)

In [ ]:
# Learnt state dynamics function

 ############ NEW #############
def fn_dynamics(x_c):
    # Brendan (3) modified to predict on the extended state
    x = tf.convert_to_tensor(x_c,dtype=float)
    dx = frozen_func(x)[0]
    dx = dx.numpy()
    return dx[:,0,:] # only return batch x state
 ############ NEW #############

Cross Entropy Method function

In [ ]:
# Cross entropy method: Finds a control signal to move the starting state to the goal state, by minimising 
# the step-wise cross entropy
#
# in:
#     x0 - initial state
#     xt - target state
#     time_step - the amount of timesteps required to converge
#     delta_time - 
#     trials - 
#     horizon_num
#     sample_num
#     epsilon
#     alpha_mean
#     alpha_var
# out:
#     X - 
#     U - optimal control signal
#     R - reward

def cross_entropy_method(initial_state,
                        target_state,
                        time_step=150, 
                        delta_time=0.2,
                        trials=50,
                        horizon_num=100,
                        sample_num=100,
                        elite_limit=10,
                        epsilon=0.1,
                        alpha_mean=0.1,
                        alpha_var=0.7):
    
    
    X = np.array([initial_state[-1]])  
    U = np.array([])
    R = np.array([])
    
    tt = list()
    
    cur_state = initial_state.copy()
    rand_center = 0
    #import pdb
    #pdb.set_trace()
    for step in range(time_step):

        
        
        start_step_time = time.time()
        lim = 1
        min_lim = rand_center-lim
        max_lim = rand_center+lim
        
        for trial in range(trials):
            #print('trial {}'.format(trial))
            u_samples = np.random.uniform(low=min_lim,
                                          high=max_lim,
                                          size=(sample_num, horizon_num))
            var_u = np.var(u_samples[:,0])
            mean_u = np.mean(u_samples[:,0])
            reward = np.zeros(sample_num)
            
            horizon_state = np.repeat([cur_state], sample_num, axis=0)
            
            for horizon_step in range(horizon_num):
                
                    ############ NEW #############
                    # Brendan (4) Add the control to the final timestep
                    horizon_state[:,-1,3]=u_samples[:,horizon_step]
                
                    # Brendan (5) Predict on the augmented state
                    print('Input: {}'.format(horizon_state))
                    horizon_state_p = fn_dynamics(horizon_state)
                    print('output: {}'.format(horizon_state_p))

                    # Brendan (6) modified the reward code to use the new state p
                    reward_function_weight = [300,100]
                    reward = reward + 1/( \
                        reward_function_weight[0]*abs(target_state[0]-abs(horizon_state_p[:,0])) + \
                        reward_function_weight[1]*abs(target_state[1]-abs(horizon_state_p[:,1]))
                                        )

                    # Brendan (7) append zero control for the next timestep, to prepare for (4) in next loop
                    horizon_state_p = np.concatenate([horizon_state_p,np.nan((sample_num,1))],axis=1)
                    # Brendan (8) remove the first timestep from the history
                    horizon_state = np.delete(horizon_state, 0, 1)                
                    # Brendan (9) add the new prediction to the history
                    horizon_state = np.concatenate([horizon_state,np.expand_dims(horizon_state_p,axis=1)],axis=1)
                    
                     ############ NEW #############

       
            reward_sort_idx = reward.argsort()
            sorted_u_samples = u_samples[reward_sort_idx[::-1]]
            sorted_rewards = reward[reward_sort_idx[::-1]]
            
            elite_u = sorted_u_samples[:elite_limit,:]
            elite_r = sorted_rewards[:elite_limit]
            
            var_u = (alpha_var*var_u)+((1-alpha_var)*np.var(elite_u[:,0]))
            mean_u = (alpha_mean*mean_u)+((1-alpha_mean)*np.mean(elite_u[:,0]))
            
            min_lim = mean_u-var_u
            max_lim = mean_u+var_u
            
            if var_u < epsilon:
                #print(trial,var_u)
                break

        
        ############ NEW #############
        # Brendan (10)  Add the control to the final timestep
        cur_state[-1,3]=elite_u[0,0]
        
        # Brendan (11) Predict on the augmented state
        cur_state_p = fn_dynamics(np.expand_dims(cur_state,axis=0))[0]

        
        # Brendan (12) append zero control for the next timestep, to prepare for (11) on next loop
        cur_state_p = np.concatenate([cur_state_p,np.zeros(1)],axis=0)
        # Brendan (13) remove the first timestep from the history
        cur_state = np.delete(cur_state, 0, 0)                
        # Brendan (14) add the new prediction to the history
        cur_state = np.concatenate([cur_state,np.expand_dims(cur_state_p,axis=0)],axis=0)
        ############ NEW #############
        
        current_time = time.time()
        time_taken = current_time-start_step_time
        print('Step', step+1,'/',time_step, ':', cur_state, ', time: ', time_taken, 'Freq:',1/(time_taken), ' Hz')

        
         ############ NEW #############
        # Brendan (15) Return only final timestep
        X = np.append(X,[cur_state[-1]], axis = 0)
         ############ NEW #############
            
        U = np.append(U,elite_u[0,0])
        R = np.append(R,elite_r[0])
        
        tt.append(time_taken)
    print(1/np.mean(np.array(tt)))
    print(1/np.var(np.array(tt)))
    return X, U, R

Run Cross Entropy Method using the defined hyperparameters and calculate running time.

In [ ]:
# Apply the cross entropy method to move the system to the target state
X, U, R = cross_entropy_method(x0,
                               xt,
                               time_step=time_step, 
                               delta_time=delta_time,
                               trials=trials,
                               horizon_num=horizon_num,
                               sample_num=sample_num,
                               elite_limit=elite_limit,
                               epsilon=epsilon,
                               alpha_mean=alpha_mean,
                               alpha_var=alpha_var)

#%lprun -f cross_entropy_method cross_entropy_method(x0,xt,time_step=time_step, delta_time=delta_time,trials=trials,horizon_num=horizon_num,sample_num=sample_num,elite_limit=elite_limit,epsilon=epsilon,alpha_mean=alpha_mean,alpha_var=alpha_var)

print("--- %s seconds ---" % (time.time() - start_time))

Output plot of the states.

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6,7), sharey=False)
ax1.plot(X[:,0])
ax1.set_title('Angle')
ax2.plot(X[:,1])
ax2.set_title('Angular velocity')
# ax3.plot(np.abs(np.sin(X[:,2]/2)))
# ax3.set_title('Angle')
# ax4.plot(X[:,3])
# ax4.set_title('Angular Velocity')
ax3.plot(U)
ax3.set_title('Control')
ax4.plot(R)
ax4.set_title('Reward')